In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pymorphy2
import re
from numba import jit
import numpy as np

In [2]:
from multiprocessing import Pool

## Описание полей
 * __id__ &mdash; внутренний идетификатор
 * __name__ &mdash; название вакансии
 * __description__ &mdash; текст вакансии
 * __target__ &mdash; класс заинтересованности

In [3]:
def get_data(processed=False):
    if processed:
        train_data = pd.read_csv('data/train_processed.csv', sep='\t', index_col='id')
        test_data = pd.read_csv('data/test_processed.csv', sep='\t', index_col='id') 
    else:
        train_data = pd.read_csv('data/train.csv', sep='\t', index_col='id')
        test_data = pd.read_csv('data/test.csv', sep='\t', index_col='id') 
    return (train_data, test_data)

In [4]:
## Токенизатор 
def get_lemmas(text):
    text = BeautifulSoup(text, 'lxml').get_text()       
    lemms = [morph.parse(word)[0].normal_form for word in text.split()]
        
    return " ".join([x for x in lemms if (re.match('^[a-zа-яё]+(-[a-zа-яё]+)*$', x) and len(x)>2)])
       
def get_vacancies_lemmas(texts):  
    texts = np.array(texts)    
    processed = np.vectorize(get_lemmas, otypes=[str])              
    return processed(texts)

In [5]:
train_data = pd.read_csv('data/train.csv', sep='\t', index_col='id')

In [6]:
%%time

morph = pymorphy2.MorphAnalyzer()

train_data['name_processed'] = get_vacancies_lemmas(train_data['name'])
train_data['description_processed'] = get_vacancies_lemmas(train_data['description'])


Wall time: 2h 49min 30s


In [7]:
train_data.to_csv('data/train_processed2.csv', '\t')